In [1]:
# 示範 backtrader 使用方法

In [1]:
import os
import backtrader as bt 
import numpy as np
import pandas as pd
import yfinance as yf

In [ ]:
# 取得當前檔案的所在目錄
current_dir = os.path.dirname(os.path.abspath(__name__))
parent_dir1 = os.path.abspath(os.path.join(current_dir, ".."))

In [4]:
current_dir

'/Users/houguanyu/Documents/code/python/stock/Quant/PythonQuantrading/Chapter1/1-4'

In [5]:
parent_dir1

'/Users/houguanyu/Documents/code/python/stock/Quant/PythonQuantrading/Chapter1'

In [ ]:
"""加載回測數據：讀取 CSV 檔案"""
# 使用 GenericCSVData 方法載入 CSV 格式的股票數據
data = bt.feeds.GenericCSVData(
    dataname=parent_dir1 + "/data/stock_data_example.csv",  # 指定 CSV 檔案的路徑
    datatime=0, # 設定 datetime 欄位的位置
    open=1,  # 設定 open 欄位的位置
    high=2,  # 設定 high 欄位的位置
    low=3,   # 設定 low 欄位的位置
    close=4, # 設定 close 欄位的位置
    volume=5,  # 設定 volume 欄位的位置
    openinterest=-1,  # 設定 open interest 欄位，這裡不使用，設為 -1
    dtformat=("%Y-%m-%d"),  # 指定日期格式
) # type: ignore
cerebro = bt.Cerebro()  # 初始化 Cerebro 引擎
cerebro.adddata(data)  # 將數據添加到 Cerebro 中
results = cerebro.run()  # 執行回測 

In [ ]:
"""加載回測數據：從 yfinance 下載資料"""
# 使用 Pandasdata 方法載入 yfinacne 股票數據
data_2330 = bt.feeds.PandasData(
    dataname=yf.download("2330.TW", "2020-01-01", "2020-01-31").droplevel(
        "Ticker", axis=1
    )[
        ["Open", "High", "Low", "Close", "Volume"]
    ]  # 下載 2330.TW 股票在指定日期範圍內的數據
)


/var/folders/jd/cst_1lwd4vdgv06j7c62ffp00000gn/T/ipykernel_84297/2798382880.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  dataname=yf.download("2330.TW", "2020-01-01", "2020-01-31").droplevel(
[*********************100%***********************]  1 of 1 completed


In [21]:
data_2317 = bt.feeds.PandasData(
    dataname=yf.download("2317.TW", "2020-01-01", "2020-01-31").droplevel(
        "Ticker", axis=1
    )[
        ["Open", "High", "Low", "Close", "Volume"]
    ]  # 下載 2317.TW 股票在指定日期範圍內的數據
)
cerebro = bt.Cerebro() # 初始化 Cerebro 引擎
cerebro.adddata(data_2330) # 將 2330.TW 股票數據添加到 Cerebro 中
cerebro.adddata(data_2317) # 將 2317.TW 股票數據添加到 Cerebro 中
cerebro.run()  # 執行回測

/var/folders/jd/cst_1lwd4vdgv06j7c62ffp00000gn/T/ipykernel_84297/409716747.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  dataname=yf.download("2317.TW", "2020-01-01", "2020-01-31").droplevel(
[*********************100%***********************]  1 of 1 completed


In [35]:
"""加載回測數據：自訂資料格式"""
# 生成一個自訂的資料集，包含日期、價格和自定義因子
data = [
    ["2020-01-02", 74.06, 75.15, 73.80, 75.09, 135480400, 1, 11],
    ["2020-01-03", 74.29, 75.14, 74.12, 74.36, 146322000, 2, 12],
    ["2020-01-06", 73.45, 74.99, 73.19, 74.95, 118387200, 3, 13],
    ["2020-01-07", 74.96, 5.22, 74.37, 74.60, 108872000, 4, 14],
    ["2020-01-08", 74.29, 76.11, 74.29, 75.80, 132079200, 5, 15],
]
data = pd.DataFrame(data)  # 將數據轉換為 DataFrame 格式 
data.columns = [  # 設定 DataFrame 的欄位名稱
    "datatime",
    "Open",
    "High",
    "Low",
    "Close",
    "Volume",
    "factor1",
    "factor2",
]
data["datetime"] = pd.to_datetime(data["datatime"])  # 將 datetime 欄位轉換為日期格式

In [36]:
data

,datatime,Open,High,Low,Close,Volume,factor1,factor2,datetime
0,2020-01-02,74.06,75.15,73.80,75.09,135480400,1,11,2020-01-02
1,2020-01-03,74.29,75.14,74.12,74.36,146322000,2,12,2020-01-03
2,2020-01-06,73.45,74.99,73.19,74.95,118387200,3,13,2020-01-06
3,2020-01-07,74.96,5.22,74.37,74.60,108872000,4,14,2020-01-07
4,2020-01-08,74.29,76.11,74.29,75.80,132079200,5,15,2020-01-08


In [37]:
# 定義包含自定義因子的 PandasData 類別
class PandasDataWithFactor(bt.feeds.PandasData):
    params = (
        ("datetime", "datetime"),  # 對應 datetime 欄位 
        ("open", "Open"),  # 對應 open 欄位
        ("high", "High"),  # 對應 high 欄位
        ("low", "Low"),  # 對應 low 欄位
        ("close", "Close"),  # 對應 close 欄位
        ("volume", "Volume"),  # 對應 volume 欄位
        ("factor1", "factor1"),  # 定義自訂因子 factor1，對應第 6 欄位
        ("factor2", "factor2"),  # 定義自訂因子 factor2，對應第 7 欄位
        ("openinterest", -1),    # 不使用 open interest，設為 1
    )

data = PandasDataWithFactor(dataname=data)
cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.run()

In [34]:
"""加載回測數據: 自訂資料格式"""
# 生成一個自訂的資料集，包含日期、價格和自定義因子
data = [
    ["2020-01-02", 74.06, 75.15, 73.80, 75.09, 135480400, 1, 11],
    ["2020-01-03", 74.29, 75.14, 74.12, 74.36, 146322800, 2, 12],
    ["2020-01-06", 73.45, 74.99, 73.19, 74.95, 118387200, 3, 13],
    ["2020-01-07", 74.96, 75.22, 74.37, 74.60, 108872000, 4, 14],
    ["2020-01-08", 74.29, 76.11, 74.29, 75.80, 132079200, 5, 15],
]
data = pd.DataFrame(data)  # 將數據轉換為 DataFrame 格式
data.columns = [  # 設定 DataFrame 的欄位名稱
    "datetime",
    "Open",
    "High",
    "Low",
    "Close",
    "Volume",
    "factor1",
    "factor2",
]
data["datetime"] = pd.to_datetime(data["datetime"])  # 將 datetime 欄位轉換為日期格式


# 定義包含自定義因子的 PandasData 類別
class PandasDataWithFactor(bt.feeds.PandasData):
    params = (
        ("datetime", "datetime"),  # 對應 datetime 欄位
        ("open", "Open"),  # 對應 open 欄位
        ("high", "High"),  # 對應 high 欄位
        ("low", "Low"),  # 對應 low 欄位
        ("close", "Close"),  # 對應 close 欄位
        ("volume", "Volume"),  # 對應 volume 欄位
        ("factor1", "factor1"),  # 定義自訂因子 factor1，對應第 6 欄位
        ("factor2", "factor2"),  # 定義自訂因子 factor2，對應第 7 欄位
        ("openinterest", -1),  # 不使用 open interest，設為 -1
    )


data = PandasDataWithFactor(dataname=data)  # 初始化 PandasDataWithFactor
cerebro = bt.Cerebro()  # 初始化 Cerebro 引擎
cerebro.adddata(data)  # 將數據添加到 Cerebro 中
cerebro.run()  # 執行回測

In [41]:
"""範例策略一：印出交易日當天和前一天的開盤價和收盤價"""

# 定義一個策略類別，印出較易日當天和前一天的開盤價和收盤價
class PrintDataStrategy(bt.Strategy):
    # next 方法會在每個時間點被執行
    def next(self):
        # self.datas[0] 代表第一個數據集（即第一支股票）
        date = self.datas[0].datetime.date(0) # 取得當前交易日的日期
        close = self.datas[0].close[0] # 取得當前交易日的收盤價
        open = self.datas[0].open[0] # 取得當前交易日的開盤價
        # len(self.datas[0]) 對應當前是第幾個交易日
        # 隨著每個交易日的進行，len(self.datas[0]) 會不斷增加
        # 每當 next() 方法被調用時，len(self.datas[0]) 值會加 1
        print(
            f"Day-{len(self.datas[0])}, "
            + f"Date: {date}, "
            + f"Open: {open}"
        )
        # 檢查數據集中是否有前一天的資料
        # 這個檢查是為了避免存取不存在的前一天資料
        if len(self.datas[0]) > 1:
            # 索引 [-1] 表示前一個時間點的數據
            # 取得前一個較易日的日期、收便價和開盤價
            yesterday_date = self.datas[0].datetime.date(-1)
            yesterday_close = self.datas[0].close[-1]
            yesterday_open = self.datas[0].open[-1]
            print(
                f"Yesterday Date: {yesterday_date}, "
                + f"Close: {yesterday_close}, "
                + f"Open: {yesterday_open}"
            )

        print("-------------------------------------------------------")
        
data = bt.feeds.GenericCSVData(
    dataname=parent_dir1 + "/data/stock_data_example.csv",  # 指定 CSV 檔案的路徑
    datetime=0,
    open=1,
    high=2,
    low=3,
    close=4,
    volume=5,
    openinterest=-1,
    dtformat=("%Y-%m-%d"),
) # type: ignore
cerebro = bt.Cerebro() # 初始化回測引擎
cerebro.adddata(data)  # 加載數據
cerebro.addstrategy(PrintDataStrategy) # 加載策略
results = cerebro.run() # 執行策略

Day-1, Date: 2020-01-02, Open: 74.06
-------------------------------------------------------
Day-2, Date: 2020-01-03, Open: 74.29
Yesterday Date: 2020-01-02, Close: 75.09, Open: 74.06
-------------------------------------------------------
Day-3, Date: 2020-01-06, Open: 73.45
Yesterday Date: 2020-01-03, Close: 74.26, Open: 74.29
-------------------------------------------------------
Day-4, Date: 2020-01-07, Open: 74.96
Yesterday Date: 2020-01-06, Close: 74.95, Open: 73.45
-------------------------------------------------------
Day-5, Date: 2020-01-08, Open: 74.29
Yesterday Date: 2020-01-07, Close: 74.6, Open: 74.96
-------------------------------------------------------


In [56]:
#
"""
範例策略二：
當收盤價 < 開盤價（收黑）時買入，
當收盤價 > 開盤價（收紅）時賣出，
當收盤價 = 開盤價時不執行操作。
"""

class OpenCloseStrategy(bt.Strategy):
    def __init__(self):
        self.order = None  # 初始化訂單狀態為 None，訂單狀態會隨著交易變更
        self.close = self.datas[0].close  # 取得第一個數據集的收盤價資料
        self.open = self.datas[0].open    # 取得第一個數據集的開盤價資料

    def log(self, txt, dt=None):
        """記錄策略日誌的函數"""
        # 如果沒有指定日期，則預設為當前交易日日期
        dt = dt or self.datas[0].datetime.date(0)
        print(f"{dt.isoformat()} {txt}")

    def notify_order(self, order):
        """訂單通知處理"""
        if order.status in [order.Submitted]:
            self.log("訂單已提交")
        if order.status in [order.Accepted]:
            self.log("訂單已接受")
        if order.status in [order.Canceled]:
            self.log("訂單已取消")
        if order.status in [order.Margin]:
            self.log("保證金不足")
        if order.status in [order.Rejected]:
            self.log("訂單被拒絕")
        if order.status in [order.Completed]:
            executed_price = np.round(order.executed.price, 3)
            executed_comm = np.round(order.executed.comm, 3)
            if order.isbuy():
                self.log(
                    "訂單完成: 買入執行, "
                    + f"價格: {executed_price}, "
                    + f"手續費 {executed_comm}"
                )
            elif order.issell():
                self.log(
                    "訂單完成: 賣出執行, "
                    + f"價格: {executed_price}, "
                    + f"手續費 {executed_comm}"
                )

    def notify_trade(self, trade):
        """交易通知處理"""
        if trade.isclosed: # 交易結束時
            trade_pnl = np.round(trade.pnl, 2)
            trade_pnlcomm = np.round(trade.pnlcomm, 2)
            self.log(f"考慮手續費的利潤: {trade_pnlcomm}")
            self.log(f"不考慮手續費的利潤: {trade_pnl}")

    def next(self):
        today_open = np.round(self.open[0], 2)
        today_close = np.round(self.close[0], 2)
        self.log(f"當前收盤價: {today_close}, " + f"當前開盤價: {today_open}")
        if self.close[0] < self.open[0]:
            # 如果收盤價小魚開盤價，表示當日收黑，則執行買入操作
            self.buy(size=1) # 買入一股
            self.log("收盤價小於開盤價, 執行買入")
        elif self.close[0] > self.open[0]:
            # 如果收盤價大於開盤價，表示當日收紅，則值性賣出操作
            self.sell(size=1) # 賣出一股
            self.log("收盤價大於開盤價, 執行賣出")

In [57]:
# 加載數據
data = bt.feeds.GenericCSVData(
    dataname=parent_dir1 + "/data/stock_data_example.csv",
    datetime=0,
    open=1,
    high=2,
    low=3, 
    close=4,
    volume=5,
    openinterest=-1,
    dtformat=("%Y-%m-%d"),
) # type: ignore
cerebro = bt.Cerebro() # 初始化回測引擎
cerebro.adddata(data)  # 加載數據
cerebro.addstrategy(OpenCloseStrategy)  # 加載測略
cerebro.broker.setcash(100000)  # 設置初始資金
cerebro.broker.setcommission(commission=0.0015)  # 設置交易手續費
results = cerebro.run() # 執行回測

2020-01-02 當前收盤價: 75.09, 當前開盤價: 74.06
2020-01-02 收盤價大於開盤價, 執行賣出
2020-01-03 訂單已提交
2020-01-03 訂單已接受
2020-01-03 訂單完成: 賣出執行, 價格: 74.29, 手續費 0.111
2020-01-03 當前收盤價: 74.26, 當前開盤價: 74.29
2020-01-03 收盤價小於開盤價, 執行買入
2020-01-06 訂單已提交
2020-01-06 訂單已接受
2020-01-06 訂單完成: 買入執行, 價格: 73.45, 手續費 0.11
2020-01-06 考慮手續費的利潤: 0.62
2020-01-06 不考慮手續費的利潤: 0.84
2020-01-06 當前收盤價: 74.95, 當前開盤價: 73.45
2020-01-06 收盤價大於開盤價, 執行賣出
2020-01-07 訂單已提交
2020-01-07 訂單已接受
2020-01-07 訂單完成: 賣出執行, 價格: 74.96, 手續費 0.112
2020-01-07 當前收盤價: 74.6, 當前開盤價: 74.96
2020-01-07 收盤價小於開盤價, 執行買入
2020-01-08 訂單已提交
2020-01-08 訂單已接受
2020-01-08 訂單完成: 買入執行, 價格: 74.29, 手續費 0.111
2020-01-08 考慮手續費的利潤: 0.45
2020-01-08 不考慮手續費的利潤: 0.67
2020-01-08 當前收盤價: 75.8, 當前開盤價: 74.29
2020-01-08 收盤價大於開盤價, 執行賣出


---

您好！您觀察到「不是每次的訂單完成都會印出利潤」這個現象，這是因為在 Backtrader 中，`notify_order` 和 `notify_trade` 這兩個方法的功能是不同的，它們在訂單和交易的生命週期中被觸發的時機也不同。

---

### 原因分析：`notify_order` 和 `notify_trade` 的區別

1.  **`notify_order(self, order)`：處理訂單的狀態變化**
    * 這個方法會在您的**訂單**狀態發生變化時被觸發。
    * 訂單狀態包括：`Submitted` (已提交), `Accepted` (已接受), `Canceled` (已取消), `Margin` (保證金不足), `Rejected` (被拒絕), `Completed` (已完成/成交)。
    * 當 `order.status in [order.Completed]` 時，表示您的**訂單已經成交**（無論是買入還是賣出）。此時您會看到訂單的成交價格和手續費，就像您輸出中顯示的：
        * `2020-01-03 訂單完成: 賣出執行, 價格: 74.29, 手續費 0.111`
        * `2020-01-06 訂單完成: 買入執行, 價格: 73.45, 手續費 0.11`
        * `2020-01-07 訂單完成: 賣出執行, 價格: 74.96, 手續費 0.112`
        * `2020-01-08 訂單完成: 買入執行, 價格: 74.29, 手續費 0.111`

2.  **`notify_trade(self, trade)`：處理交易的生命週期和損益結算**
    * 這個方法會在**一筆完整的交易（買入並隨後賣出，或賣出並隨後買入，即平倉）**完成時被觸發。
    * 一筆「交易」的概念是，您先開倉（買入或賣出），然後再平倉（賣出或買入以關閉該倉位）。只有當這個完整的「開倉-平倉」循環結束後，Backtrader 才會產生一筆 `trade` 對象，並計算其最終損益。
    * `trade.isclosed` 為 `True` 時，表示這筆交易已經完全平倉了，這時候才能計算出最終的損益 (profit and loss, PnL)。

### 為什麼您的輸出不是每次訂單完成都印出利潤？

結合您的策略和輸出，原因如下：

* **您的策略是每次都買入或賣出 `size=1` 股。**
* **Backtrader 預設是 FIFO (先進先出) 原則來管理倉位。**
* **利潤只在交易平倉時才計算和顯示。**

讓我們逐步分析您的日誌：

1.  **`2020-01-02 當前收盤價: 75.09, 當前開盤價: 74.06`**
    * `收盤價大於開盤價, 執行賣出`：您下了一個賣出訂單 (`self.sell(size=1)` )。此時，您手上應該是**沒有持股**的，所以這是一個**開空倉**的動作。

2.  **`2020-01-03 訂單完成: 賣出執行, 價格: 74.29, 手續費 0.111`**
    * 賣出訂單成交了。您現在持有 -1 股（即做空了 1 股）。
    * `2020-01-03 當前收盤價: 74.26, 當前開盤價: 74.29`
    * `收盤價小於開盤價, 執行買入`：您下了一個買入訂單 (`self.buy(size=1)` )。由於您之前是空頭倉位 (-1 股)，這次買入 1 股會**平掉您的空倉**。

3.  **`2020-01-06 訂單已提交` 到 `2020-01-06 訂單完成: 買入執行, 價格: 73.45, 手續費 0.11`**
    * 買入訂單成交了。您的空倉被平掉了。
    * **此時，由於一筆完整的交易 (賣出 -> 買入) 完成了，`notify_trade` 被觸發。**
    * **`2020-01-06 考慮手續費的利潤: 0.62`**
    * **`2020-01-06 不考慮手續費的利潤: 0.84`**
    * 這是您第一次看到利潤，因為這是第一筆被完全平倉的交易。

4.  **`2020-01-06 當前收盤價: 74.95, 當前開盤價: 73.45`**
    * `收盤價大於開盤價, 執行賣出`：您再次下了一個賣出訂單 (`self.sell(size=1)` )。由於您剛剛平倉，現在您手上又**沒有持股**了，所以這又是一個**開空倉**的動作。

5.  **`2020-01-07 訂單已提交` 到 `2020-01-07 訂單完成: 賣出執行, 價格: 74.96, 手續費 0.112`**
    * 賣出訂單成交了。您現在持有 -1 股。
    * `2020-01-07 當前收盤價: 74.6, 當前開盤價: 74.96`
    * `收盤價小於開盤價, 執行買入`：您下了一個買入訂單 (`self.buy(size=1)` )。這會**平掉您的空倉**。

6.  **`2020-01-08 訂單已提交` 到 `2020-01-08 訂單完成: 買入執行, 價格: 74.29, 手續費 0.111`**
    * 買入訂單成交了。您的空倉被平掉了。
    * **再次，由於一筆完整的交易 (賣出 -> 買入) 完成了，`notify_trade` 被觸發。**
    * **`2020-01-08 考慮手續費的利潤: 0.45`**
    * **`2020-01-08 不考慮手續費的利潤: 0.67`**
    * 您第二次看到利潤。

### 結論

因此，您的程式碼行為是完全符合 Backtrader 設計的。利潤信息只會在 **一筆完整的交易（開倉並平倉）結束時** 透過 `notify_trade` 函數列印出來，而不是在每次單獨的買入或賣出訂單完成時就列印。

這在實際交易中也是合理的，因為只有在平倉後，您才能真正確定這筆交易的最終損益。

In [ ]:
"""範例策略三：每月定期定額0050"""

class MontylyInvestmentStrategy(bt.Strategy):
    # 定義策略的參數
    params = (
        ("cash_to_invest", None), # 每月計劃投資的金額
        ("investment_day", None), # 每月的投資日（哪一天執行定期投資）
    )

    def __init__(self):
        # 初始化一個定時器，設置在每個月的指定投資日執行操作
        self.add_timer(
            when=bt.Timer.SESSION_START, # 在指定交易日開始時觸發
            monthdays=[self.params.investment_day], # 每個月的指定投資日 # type: ignore
        )

        # 初始化訂單狀態為 None，用來追蹤當前的訂單
        self.order = None

    def log(self, txt, dt=None):
        """記錄策略日誌的函數"""
        dt = dt or self.datas[0].datetime.date(0)
        print(f"{dt.isoformat()} {txt}")

    def notify_order(self, order):
        """訂單通知處理"""
        if order.status in [order.Completed]:
            # 取得成交價格
            executed_price = np.round(order.executed.price, 2)
            # 取得手續費
            executed_comm = np.round(order.executed.comm, 2)
            self.log(
                "訂單完成: 買入執行,"
                + f"價格: {executed_price}, 手續費 {executed_comm}"
            )

    def notify_timer(self, timer, when, *args, **kwargs):
        """定時器觸發時執行的操作"""
        self.log("定投日, 執行買入訂單")
        # 取得當前帳戶可用的現金金額
        cash_available = self.broker.getcash()
        # 取得當前收盤價
        price = self.data.close[0]
        # 根據每月計劃投資的金額 (self.params.chash_to_invest)和可用的現金
        # 計算可以買入的股票數量 size
        size = min(self.params.cash_to_invest, cash_available) // price
        if size > 0:
            # 如果計算出可以買的股票數量大於0，則執行買入訂單
            self.order = self.buy(size=size)
    
    # next 是在每個交易日都會被使用的方法，但在這個範例中不做任何操作，
    # 全部交由定時器控制交易
    def next(self):
        pass

cerebro = bt.Cerebro() # 初始化回測引擎
# 加載數據
asset = "0050.TW"
data = bt.feeds.PandasData(
    dataname=yf.download(asset, "2011-01-01", "2020-12-31").droplevel("Ticker", axis=1)[
        ["Open", "High", "Low", "Close", "Volume"]
    ]
)
cerebro.adddata(data)
# 加載策略（每個月投資一萬且交易日射潿每個月的第一個交易日）
cerebro.addstrategy(MontylyInvestmentStrategy, cash_to_invest=10000, investment_day=1)
initial_cash = 10 * 12 * 10000 # 初始資金
cerebro.broker.setcash(initial_cash)
cerebro.broker.setcommission(commission=0.0015) # 設置交易手續費
# 加載分析氣
cerebro.addanalyzer(bt.analyzers.SharpeRatio)
cerebro.addanalyzer(bt.analyzers.DrawDown)
cerebro.addanalyzer(bt.analyzers.Returns)
cerebro.addanalyzer(bt.analyzers.PyFolio)
# 執行回測
results = cerebro.run()
strat = results[0]
# 輸出分析器的結果
print(f"Sharp Ratio: {strat.analyzers.sharperatio.get_analysis()}")
print(f"Drawdown: {strat.analyzers.drawdown.get_analysis()}")
print(f"Returns: ", strat.analyzers.returns.get_analysis())
# 輸出 broker 內資訊
print("----------------------------------------------")
print(f"股票代碼: {asset}")
positions = cerebro.broker.getposition(data)
print(f"總投資股數: {positions.size}")
final_portfolio_value = cerebro.broker.getvalue()
print(f"總資產(包含剩餘現金): {final_portfolio_value:.2f}")
profit = (final_portfolio_value - initial_cash) / initial_cash
print(f"總收益率: {profit * 100:.2f}%")
final_cash = cerebro.broker.getcash()
print(f"最終現金餘額: {final_cash:.2f}")

/var/folders/jd/cst_1lwd4vdgv06j7c62ffp00000gn/T/ipykernel_84297/2549257973.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  dataname=yf.download(asset, "2011-01-01", "2020-12-31").droplevel("Ticker", axis=1)[
[*********************100%***********************]  1 of 1 completed


2011-01-03 定投日, 執行買入訂單
2011-01-04 訂單完成: 買入執行,價格: 47.24, 手續費 15.02
2011-02-08 定投日, 執行買入訂單
2011-02-09 訂單完成: 買入執行,價格: 47.97, 手續費 14.97
2011-03-01 定投日, 執行買入訂單
2011-03-02 訂單完成: 買入執行,價格: 45.63, 手續費 14.85
2011-04-01 定投日, 執行買入訂單
2011-04-06 訂單完成: 買入執行,價格: 46.05, 手續費 14.99
2011-05-03 定投日, 執行買入訂單
2011-05-04 訂單完成: 買入執行,價格: 47.28, 手續費 14.82
2011-06-01 定投日, 執行買入訂單
2011-06-02 訂單完成: 買入執行,價格: 47.59, 手續費 14.85
2011-07-01 定投日, 執行買入訂單
2011-07-04 訂單完成: 買入執行,價格: 45.98, 手續費 15.03
2011-08-01 定投日, 執行買入訂單
2011-08-02 訂單完成: 買入執行,價格: 45.98, 手續費 14.83
2011-09-01 定投日, 執行買入訂單
2011-09-02 訂單完成: 買入執行,價格: 42.33, 手續費 14.99
2011-10-03 定投日, 執行買入訂單
2011-10-04 訂單完成: 買入執行,價格: 38.3, 手續費 14.88
2011-11-01 定投日, 執行買入訂單
2011-11-02 訂單完成: 買入執行,價格: 39.61, 手續費 14.67
2011-12-01 定投日, 執行買入訂單
2011-12-02 訂單完成: 買入執行,價格: 38.46, 手續費 14.94
2012-01-02 定投日, 執行買入訂單
2012-01-03 訂單完成: 買入執行,價格: 37.99, 手續費 15.1
2012-02-01 定投日, 執行買入訂單
2012-02-02 訂單完成: 買入執行,價格: 40.45, 手續費 15.11
2012-03-01 定投日, 執行買入訂單
2012-03-02 訂單完成: 買入執行,價格: 42.98, 手續費 15.02
2012-04-02 定

In [65]:
import yfinance as yf
import pandas as pd

ticker = "0050.TW"  # 元大台灣50在Yahoo Finance上的代碼
data = yf.download(ticker, start="2011-01-04", end="2011-01-05") # 查詢一天，結束日期設定為隔天
print(data['Close'])
print(data['Open'])
print(data['High'])

/var/folders/jd/cst_1lwd4vdgv06j7c62ffp00000gn/T/ipykernel_84297/3034993263.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2011-01-04", end="2011-01-05") # 查詢一天，結束日期設定為隔天
[*********************100%***********************]  1 of 1 completed

Ticker        0050.TW
Date                 
2011-01-04  44.233913
Ticker        0050.TW
Date                 
2011-01-04  44.414313
Ticker        0050.TW
Date                 
2011-01-04  44.450392


In [63]:
strat.analyzers.sharperatio.get_analysis()

OrderedDict([('sharperatio', 0.25951220947595965)])

In [66]:
dataname=yf.download(asset, "2011-01-01", "2020-12-31").droplevel("Ticker", axis=1)[
        ["Open", "High", "Low", "Close", "Volume"]
]

/var/folders/jd/cst_1lwd4vdgv06j7c62ffp00000gn/T/ipykernel_84297/2583321103.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  dataname=yf.download(asset, "2011-01-01", "2020-12-31").droplevel("Ticker", axis=1)[
[*********************100%***********************]  1 of 1 completed


In [67]:
dataname

Price,Open,High,Low,Close,Volume
Date,,,,,
2011-01-03,47.282370,47.320751,47.167236,47.167236,12025
2011-01-04,47.243994,47.282372,47.052101,47.052101,11080
2011-01-05,46.898587,47.090480,45.977504,46.284531,20742
2011-01-06,46.246153,46.284531,46.015882,46.207775,6872
2011-01-07,46.246155,46.322911,45.593721,45.785614,12254
...,...,...,...,...,...
2020-12-24,28.254308,28.361693,28.206580,28.349762,9804560
2020-12-25,28.421351,28.540668,28.302034,28.385555,11205268
2020-12-28,28.409420,28.636122,28.409420,28.636122,13629768


In [77]:
data1 = yf.download(asset, "2011-01-01", "2020-12-31", auto_adjust=False)
print(data1)

[*********************100%***********************]  1 of 1 completed

Price       Adj Close      Close       High        Low       Open    Volume
Ticker        0050.TW    0050.TW    0050.TW    0050.TW    0050.TW   0050.TW
Date                                                                       
2011-01-03  47.167236  61.450001  61.650002  61.450001  61.599998     12025
2011-01-04  47.052101  61.299999  61.599998  61.299999  61.549999     11080
2011-01-05  46.284531  60.299999  61.349998  59.900002  61.099998     20742
2011-01-06  46.207775  60.200001  60.299999  59.950001  60.250000      6872
2011-01-07  45.785614  59.650002  60.349998  59.400002  60.250000     12254
...               ...        ...        ...        ...        ...       ...
2020-12-24  28.349762  29.700001  29.712500  29.549999  29.600000   9804560
2020-12-25  28.385555  29.737499  29.900000  29.650000  29.775000  11205268
2020-12-28  28.636122  30.000000  30.000000  29.762501  29.762501  13629768
2020-12-29  28.612259  29.975000  30.049999  29.837500  30.012501  12375916
2020-12-30  

In [69]:
data

Price,Close,High,Low,Open,Volume
Ticker,0050.TW,0050.TW,0050.TW,0050.TW,0050.TW
Date,,,,,
2011-01-03,47.167236,47.320751,47.167236,47.282370,12025
2011-01-04,47.052101,47.282372,47.052101,47.243994,11080
2011-01-05,46.284531,47.090480,45.977504,46.898587,20742
2011-01-06,46.207775,46.284531,46.015882,46.246153,6872
2011-01-07,45.785614,46.322911,45.593721,46.246155,12254
...,...,...,...,...,...
2020-12-24,28.349762,28.361693,28.206580,28.254308,9804560
2020-12-25,28.385555,28.540668,28.302034,28.421351,11205268


In [70]:
asset

'0050.TW'

In [78]:
data2 = yf.download(ticker, start="2011-01-04", end="2011-01-05", auto_adjust=False)
print(data2)

[*********************100%***********************]  1 of 1 completed

Price       Adj Close      Close       High        Low       Open  Volume
Ticker        0050.TW    0050.TW    0050.TW    0050.TW    0050.TW 0050.TW
Date                                                                     
2011-01-04  44.233913  61.299999  61.599998  61.299999  61.549999   11080


In [ ]:
data2

Price,Close,High,Low,Open,Volume
Ticker,0050.TW,0050.TW,0050.TW,0050.TW,0050.TW
Date,,,,,
2011-01-04,44.233913,44.450392,44.233913,44.414313,11080


In [79]:
import yfinance as yf

asset = "0050.TW"
# 多天資料，禁用還原股價
data1 = yf.download(asset, "2011-01-01", "2020-12-31", auto_adjust=False)
print(data1)

# 單天資料，禁用還原股價
data2 = yf.download(asset, start="2011-01-04", end="2011-01-05", auto_adjust=False)
print(data2)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Price       Adj Close      Close       High        Low       Open    Volume
Ticker        0050.TW    0050.TW    0050.TW    0050.TW    0050.TW   0050.TW
Date                                                                       
2011-01-03  47.167236  61.450001  61.650002  61.450001  61.599998     12025
2011-01-04  47.052101  61.299999  61.599998  61.299999  61.549999     11080
2011-01-05  46.284531  60.299999  61.349998  59.900002  61.099998     20742
2011-01-06  46.207775  60.200001  60.299999  59.950001  60.250000      6872
2011-01-07  45.785614  59.650002  60.349998  59.400002  60.250000     12254
...               ...        ...        ...        ...        ...       ...
2020-12-24  28.349762  29.700001  29.712500  29.549999  29.600000   9804560
2020-12-25  28.385555  29.737499  29.900000  29.650000  29.775000  11205268
2020-12-28  28.636122  30.000000  30.000000  29.762501  29.762501  13629768
2020-12-29  28.612259  29.975000  30.049999  29.837500  30.012501  12375916
2020-12-30  

In [ ]:
"""範例四：移動平均交叉策略"""
class MovingAverageCrossStrategy(bt.Strategy):
    # 定義策略的參數: 短期移動平均線週期和長期移動平均線週期
    params = (("short_period", 5), ("long_period", 20))
    def __init__(self):
        # 5 日均線作為短期移動平均線
        self.short_sma = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.short_period
        )
        # 20 日均線作為長期移動平均線
        self.long_sma = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.long_period
        )
        # 初始化訂單狀態, 用來追蹤當前的訂單
        self.order = None
    
    def next(self):
        if not self.position: # 當下沒有任何持倉
            # 如果短期一度平均線高於長期移動平均線，則買入股票
            if self.short_sma > self.long_sma:
                self.buy()
        else: # 當下已經有倉位
            # 如果短期移動平均線低於長期移動平均線，則賣出股票
            if self.short_sma < self.long_sma:
                self.order = self.sell() # 賣出股票

    def log(self, txt, dt=None):
        """記錄策略日誌的函數"""
        # 如果沒有指定日期，則預設為當前交易日日期
        dt = dt or self.datas[0].datetime.date(0)
        print(f"{dt.isoformat()} {txt}")

    def notify_order(self, order):
        """訂單通知處理"""
        if order.status in [order.Completed]:
            executed_price = np.round(order.executed.price, 3)
            executed_comm = np.round(order.executed.comm, 3)
            if order.isbuy():
                self.log(
                    "訂單完成: 買入執行,"
                    + f"價格: {executed_price}"
                    + f"手續費: {executed_comm}"
                )
            elif order.issell():
                self.log(
                    "訂單完成: 賣出執行," 
                    + f"價格: {executed_price}"
                    + f"手續費 {executed_comm}"
                )

cerebro = bt.Cerebro()
cerebro.addstrategy(MovingAverageCrossStrategy)
asset = "2330.TW"
data = bt.feeds.PandasData(
    dataname=yf.download(asset, "2021-01-01", "2021-12-31").droplevel("Ticker", axis=1)[
        ["Open", "High", "Low", "Close", "Volume"]
    ]
)
cerebro.adddata(data)
cerebro.broker.setcash(1000000)
cerebro.broker.setcommission(commission=0.0015)
result = cerebro.run()

strat = results[0]
# 輸出分析器的結果
print(f"Sharp Ratio: {strat.analyzers.sharperatio.get_analysis()}")
print(f"Drawdown: {strat.analyzers.drawdown.get_analysis()}")
print(f"Returns: ", strat.analyzers.returns.get_analysis())
# 輸出 broker 內資訊
print("----------------------------------------------")
print(f"股票代碼: {asset}")
positions = cerebro.broker.getposition(data)
print(f"總投資股數: {positions.size}")
final_portfolio_value = cerebro.broker.getvalue()
print(f"總資產(包含剩餘現金): {final_portfolio_value:.2f}")
profit = (final_portfolio_value - initial_cash) / initial_cash
print(f"總收益率: {profit * 100:.2f}%")
final_cash = cerebro.broker.getcash()
print(f"最終現金餘額: {final_cash:.2f}")
        

/var/folders/jd/cst_1lwd4vdgv06j7c62ffp00000gn/T/ipykernel_84297/4112179874.py:54: FutureWarning: YF.download() has changed argument auto_adjust default to True
  dataname=yf.download(asset, "2021-01-01", "2021-12-31").droplevel("Ticker", axis=1)[
[*********************100%***********************]  1 of 1 completed

2021-02-01 訂單完成: 買入執行,價格: 546.116手續費: 0.819
2021-03-02 訂單完成: 賣出執行,價格: 569.979手續費 0.855
2021-04-08 訂單完成: 買入執行,價格: 558.489手續費: 0.838
2021-04-26 訂單完成: 賣出執行,價格: 557.568手續費 0.836
2021-05-03 訂單完成: 買入執行,價格: 548.352手續費: 0.823
2021-05-04 訂單完成: 賣出執行,價格: 541.901手續費 0.813
2021-05-28 訂單完成: 買入執行,價格: 540.979手續費: 0.811
2021-06-25 訂單完成: 賣出執行,價格: 553.404手續費 0.83
2021-07-14 訂單完成: 買入執行,價格: 567.285手續費: 0.851
2021-07-22 訂單完成: 賣出執行,價格: 545.075手續費 0.818
2021-08-06 訂單完成: 買入執行,價格: 551.553手續費: 0.827
2021-08-17 訂單完成: 賣出執行,價格: 540.448手續費 0.811
2021-08-31 訂單完成: 買入執行,價格: 558.956手續費: 0.838
2021-09-23 訂單完成: 賣出執行,價格: 546.626手續費 0.82
2021-10-20 訂單完成: 買入執行,價格: 560.57手續費: 0.841
2021-11-30 訂單完成: 賣出執行,價格: 554.992手續費 0.832
2021-12-09 訂單完成: 買入執行,價格: 563.359手續費: 0.845
2021-12-10 訂單完成: 賣出執行,價格: 562.43手續費 0.844
2021-12-21 訂單完成: 買入執行,價格: 557.548手續費: 0.836
Sharp Ratio: OrderedDict({'sharperatio': 0.25951220947595965})
Drawdown: AutoOrderedDict({'len': 0, 'drawdown': 0.0, 'moneydown': 0.0, 'max': AutoOrderedDict({'len': 1421, 'draw

In [92]:
"""範例策略五：RSI 過度買入/賣出策略"""

class RSIStrategy(bt.Strategy):
    # 定義策略的參數：rsi_period, rsi_low, rsi_high
    # rsi_period: RSI 的計算週期, 預設為 14 天
    # rsi_low: 當 RSI 低於此述職時進行買入,預設為30
    # rsi_high: 當 RSI 高於此數值時進行賣出,預設為70
    params = (("rsi_period", 14), ("rsi_low", 30), ("rsi_high", 70))

    def __init__(self):
        # 使用參數中設定的 rsi_period 來計算 RSI
        self.rsi = bt.indicators.RelativeStrengthIndex(period=self.params.rsi_period)
        # 初始化訂單狀態,用來追嗡當前的訂單
        self.order = None
    
    def next(self):
        if not self.position: # 當下沒有任何持倉
            # 如果 RSI 低於設定的 rsi_low 參數值（預設為30），表示市場超賣，執行買入操作
            if self.rsi < self.params.rsi_low:
                self.buy()
        else: # 當下已經有倉位
            # 如果 RSI 高於設定的 rsi_high 參數值（預設為70），表是是場超買，執行賣出操作
            if self.rsi > self.params.rsi_high:
                self.sell()

    def log(self, txt, dt=None):
        """記錄策略日誌的函數"""
        # 如果沒有指定日期，則預設為當前交易日日期
        dt = dt or self.datas[0].datetime.date(0)
        print(f"{dt.isoformat()} {txt}")

    def notify_order(self, order):
        """訂單通知處理"""
        if order.status in [order.Completed]:
            executed_price = np.round(order.executed.price, 3)
            executed_comm = np.round(order.executed.comm, 3)
            if order.isbuy():
                self.log(
                    "訂單完成: 買入執行, "
                    + f"價格: {executed_price}, "
                    + f"手續費 {executed_comm}"
                )
            elif order.issell():
                self.log(
                    "訂單完成: 賣出執行, "
                    + f"價格: {executed_price}"
                    + f"手續費 {executed_comm}"
                )

cerebro = bt.Cerebro()
cerebro.addstrategy(RSIStrategy)
asset = "0050.TW"
data = bt.feeds.PandasData(
    dataname=yf.download(asset, "2011-01-01", "2020-12-31").droplevel("Ticker", axis=1)[
        ["Open", "High", "Low", "Close", "Volume"]
    ]
)
cerebro.adddata(data)
initial_cash = 100000
cerebro.broker.setcash(initial_cash)
cerebro.broker.setcommission(commission=0.0015)
result = cerebro.run()

strat = results[0]
# 輸出分析器的結果
print(f"Sharp Ratio: {strat.analyzers.sharperatio.get_analysis()}")
print(f"Drawdown: {strat.analyzers.drawdown.get_analysis()}")
print(f"Returns: ", strat.analyzers.returns.get_analysis())
# 輸出 broker 內資訊
print("----------------------------------------------")
print(f"股票代碼: {asset}")
positions = cerebro.broker.getposition(data)
print(f"總投資股數: {positions.size}")
final_portfolio_value = cerebro.broker.getvalue()
print(f"總資產(包含剩餘現金): {final_portfolio_value:.2f}")
profit = (final_portfolio_value - initial_cash) / initial_cash
print(f"總收益率: {profit * 100:.2f}%")
final_cash = cerebro.broker.getcash()
print(f"最終現金餘額: {final_cash:.2f}")
        

/var/folders/jd/cst_1lwd4vdgv06j7c62ffp00000gn/T/ipykernel_84297/77182208.py:54: FutureWarning: YF.download() has changed argument auto_adjust default to True
  dataname=yf.download(asset, "2011-01-01", "2020-12-31").droplevel("Ticker", axis=1)[
[*********************100%***********************]  1 of 1 completed


2011-08-08 訂單完成: 買入執行, 價格: 41.679, 手續費 0.063
2012-02-06 訂單完成: 賣出執行, 價格: 41.065手續費 0.062
2012-05-17 訂單完成: 買入執行, 價格: 38.57, 手續費 0.058
2012-08-10 訂單完成: 賣出執行, 價格: 40.566手續費 0.061
2012-10-25 訂單完成: 買入執行, 價格: 39.108, 手續費 0.059
2012-12-06 訂單完成: 賣出執行, 價格: 41.257手續費 0.062
2013-06-24 訂單完成: 買入執行, 價格: 40.988, 手續費 0.061
2014-05-29 訂單完成: 賣出執行, 價格: 12.511手續費 0.019
2014-09-30 訂單完成: 買入執行, 價格: 12.826, 手續費 0.019
2014-10-29 訂單完成: 賣出執行, 價格: 14.025手續費 0.021
2015-08-13 訂單完成: 買入執行, 價格: 13.775, 手續費 0.021
2016-03-04 訂單完成: 賣出執行, 價格: 14.083手續費 0.021
2016-05-05 訂單完成: 買入執行, 價格: 13.456, 手續費 0.02
2016-06-08 訂單完成: 賣出執行, 價格: 14.553手續費 0.022
2017-12-07 訂單完成: 買入執行, 價格: 18.612, 手續費 0.028
2018-01-19 訂單完成: 賣出執行, 價格: 19.793手續費 0.03
2018-02-07 訂單完成: 買入執行, 價格: 18.646, 手續費 0.028
2018-07-27 訂單完成: 賣出執行, 價格: 19.669手續費 0.03
2018-10-12 訂單完成: 買入執行, 價格: 17.958, 手續費 0.027
2019-03-25 訂單完成: 賣出執行, 價格: 18.051手續費 0.027
2019-05-24 訂單完成: 買入執行, 價格: 17.808, 手續費 0.027
2019-09-19 訂單完成: 賣出執行, 價格: 19.634手續費 0.029
2020-03-13 訂單完成: 買入執行, 價格: 18.613, 手

In [2]:
class MovingAverageCrossStrategy(bt.Strategy):
    # 定義策略的參數: 短期的移動平均線週期和長期的移動平均線週期
    params = (("short_period", 5), ("long_period", 20))

    def __init__(self):
        # 5日均線作為短期移動平均線
        self.short_sma = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.short_period
        )
        # 20日均線作為長期移動平均線
        self.long_sma = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.long_period
        )
        # 初始化訂單狀態, 用來追蹤當前訂單
        self.order = None

    def next(self):
        if not self.position: # 如果當下沒有任何持倉
            # 如果短期一定平均線高於長期移動平均線，則買入股票
            if self.short_sma > self.long_sma:
                self.buy() # 買入股票
        else: # 當下已經有持倉
            # 如果短期移動平均線低於長期移動平均線，則賣出股票
            if self.short_sma < self.long_sma:
                self.order = self.sell() # 賣出股票

    def log(self, txt, dt=None):
        """記錄策略日誌的函數"""
        # 如果沒有指定日期，則預設為當前交易日日期
        dt = dt or self.datas[0].datetime.date(0)
        print(f"{dt.isoformat()} {txt}")

    def notify_order(self, order):
        """訂單通知處理"""
        if order.status in [order.Completed]:
            executed_price = np.round(order.executed.price, 3)
            executed_comm = np.round(order.executed.comm, 3)
            if order.isbuy():
                self.log(
                    "訂單完成: 買入執行, "
                    + f"價格: {executed_price}"
                    + f"手續費: {executed_comm}"
                )
            elif order.issell():
                self.log(
                    "訂單完成: 賣出執行, "
                    + f"價格: {executed_price}"
                    + f"手續費: {executed_comm}"
                )

cerebro = bt.Cerebro()
cerebro.addstrategy(MovingAverageCrossStrategy)
asset = "2330.TW"
data = bt.feeds.PandasData(
    dataname=yf.download(asset, "2021-01-01", "2021-12-31").droplevel("Ticker", axis=1)[
        ["Open", "High", "Low", "Close", "Volume"]
    ]
)
cerebro.adddata(data)
initial_cash = 1000000
cerebro.broker.setcash(initial_cash)
cerebro.broker.setcommission(commission=0.0015)
results = cerebro.run()
        
strat = results[0]
# 輸出分析器的結果
# print(f"Sharp Ratio: {strat.analyzers.sharperatio.get_analysis()}")
# print(f"Drawdown: {strat.analyzers.drawdown.get_analysis()}")
# print(f"Returns: ", strat.analyzers.returns.get_analysis())
# 輸出 broker 內資訊
print("----------------------------------------------")
print(f"股票代碼: {asset}")
positions = cerebro.broker.getposition(data)
print(f"總投資股數: {positions.size}")
final_portfolio_value = cerebro.broker.getvalue()
print(f"總資產(包含剩餘現金): {final_portfolio_value:.2f}")
profit = (final_portfolio_value - initial_cash) / initial_cash
print(f"總收益率: {profit * 100:.2f}%")
final_cash = cerebro.broker.getcash()
print(f"最終現金餘額: {final_cash:.2f}")
        


/var/folders/jd/cst_1lwd4vdgv06j7c62ffp00000gn/T/ipykernel_3939/3111705395.py:55: FutureWarning: YF.download() has changed argument auto_adjust default to True
  dataname=yf.download(asset, "2021-01-01", "2021-12-31").droplevel("Ticker", axis=1)[
[*********************100%***********************]  1 of 1 completed

2021-02-01 訂單完成: 買入執行, 價格: 543.94手續費: 0.816
2021-03-02 訂單完成: 賣出執行, 價格: 567.709手續費: 0.852
2021-04-08 訂單完成: 買入執行, 價格: 556.264手續費: 0.834
2021-04-23 訂單完成: 賣出執行, 價格: 543.413手續費: 0.815
2021-05-03 訂單完成: 買入執行, 價格: 546.167手續費: 0.819
2021-05-04 訂單完成: 賣出執行, 價格: 539.742手續費: 0.81
2021-05-28 訂單完成: 買入執行, 價格: 538.824手續費: 0.808
2021-06-25 訂單完成: 賣出執行, 價格: 551.199手續費: 0.827
2021-07-14 訂單完成: 買入執行, 價格: 565.025手續費: 0.848
2021-07-22 訂單完成: 賣出執行, 價格: 542.903手續費: 0.814
2021-08-06 訂單完成: 買入執行, 價格: 549.355手續費: 0.824
2021-08-17 訂單完成: 賣出執行, 價格: 538.294手續費: 0.807
2021-08-31 訂單完成: 買入執行, 價格: 556.729手續費: 0.835
2021-09-23 訂單完成: 賣出執行, 價格: 544.448手續費: 0.817
2021-10-20 訂單完成: 買入執行, 價格: 558.337手續費: 0.838
2021-11-30 訂單完成: 賣出執行, 價格: 552.781手續費: 0.829
2021-12-09 訂單完成: 買入執行, 價格: 561.115手續費: 0.842
2021-12-10 訂單完成: 賣出執行, 價格: 560.189手續費: 0.84
2021-12-21 訂單完成: 買入執行, 價格: 555.326手續費: 0.833
----------------------------------------------
股票代碼: 2330.TW
總投資股數: 1
總資產(包含剩餘現金): 999965.96
總收益率: -0.00%
最終現金餘額: 999393.89
